In [1]:
# !pip install pandas

In [2]:
# Import libraries
import spacy
from nltk.corpus import stopwords
from collections import Counter
import nltk
from scispacy.linking import EntityLinker
import scispacy
from tqdm import tqdm
import re
import unicodedata
import os
import logging
import chardet
from collections import OrderedDict
import pandas as pd



%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# # nltk.download('punkt')
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

nlp = spacy.load("en_core_sci_md")
nlp.add_pipe("scispacy_linker", last=True)



print("Setup completed.")

/home/skanda/.local/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/home/skanda/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/skanda/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-lim

Setup completed.


## **Text Preprocessing**

## **Text Cleaning**

In [4]:
def clean_pdf_text(text):
    # Reassemble hyphenated words
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    text = re.sub(r'^[A-Z][a-zA-Z\s]+[0-9]{4};[0-9]+:[0-9]+–[0-9]+$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*\d+\s*\|\s*\d+\s*$', '', text, flags=re.MULTILINE)  # Page numbers
    text = re.sub(r'^(https?:\/\/)?(www\.)?[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,}(\/\S*)?$', '', text, flags=re.MULTILINE)  # URLs

    # Remove short lines that are likely headers or footers (less than 50 characters)
    text = re.sub(r'^\s*[\w\s]{1,50}\s*$', '', text, flags=re.MULTILINE)

    # Replace newlines that are not followed by another newline with a space
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Replace multiple newlines with a single newline to preserve paragraphs
    text = re.sub(r'\n{2,}', '\n\n', text)

    # Normalize spaces
    text = re.sub(r'\s{2,}', ' ', text)  # Collapse multiple spaces into one
    return text

def remove_urls(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, re.IGNORECASE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    return text

def remove_emails(text):
    # Pattern to match inline email addresses and lines starting with "E-mail:"
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    email_line_pattern = r'^E-mail:.*$'
    
    # Remove lines that start with "E-mail:"
    text = re.sub(email_line_pattern, '', text, flags=re.IGNORECASE)

    # Remove inline email addresses
    text = re.sub(email_pattern, '', text, flags=re.IGNORECASE)
    return text



def remove_figures_tables(text):
    return re.sub(r'\b(figures?|tables?)\b', '', text)

# def remove_numerical_references(text):
#     return re.sub(r'\[\d+\]', '', text)

def remove_citations(text):
    # Define patterns to match typical citation formats
    patterns = [
        r'\b[A-Z][a-z]+\s+[A-Z][a-z]+\s+[A-Z][a-z]+\s+et\s+al\.',  # Matches "Smith John Doe et al."
        r'\b[A-Z][a-z]+\s+[A-Z][a-z]+\s+et\s+al\.',  # Matches "Smith John et al."
        r'\b[A-Z][a-z]+\s+et\s+al\.',  # Matches "Smith et al."
        r'\(\s*et\s+al\.\s*,?\s*\d{4}\s*\)',  # Matches "(et al., 2024)"
        r'\[\d+\]',  # Matches "[1]"
        r'\(\d{4}[a-z]?(?:,\s*\d{4}[a-z]?)*\)',  # Matches "(2024, 2025a)"
        r'\b(?:19|20)\d{2}[a-z]?\b',  # Matches years like "2024" or "2024a"
    ]

    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.MULTILINE | re.IGNORECASE)
    return text

# def remove_headers(text):
#     # Remove lines that are all uppercase and end with a colon
#     text = re.sub(r'^[A-Z\s]+:$', '', text, flags=re.MULTILINE| re.IGNORECASE)
#     # Remove lines that start with bullet points
#     text = re.sub(r'^\s*•.*$', '', text, flags=re.MULTILINE| re.IGNORECASE)
#     return text

# def remove_metadata(text):
#     # Remove headers, copyright info, DOI, received/accepted dates
#     patterns = [
#         r'^.*?©Copyright.*$',
#         r'^DOI:.*$',
#         r'^Received:.*$',
#         r'^Accepted:.*$',
#         r'^Address for Correspondence:.*$',

#         r'^ORCID-ID:.*$',
#         r'^\s*\d+\s*$',  # Page numbers
#         r'^.*?ORCID:.*$',
#         r'^Cite this article as:.*$',
#         r'This is an open access article.*$',
#         r'^.*?\d{4};\d+:\d+–\d+', 
#         r'\[\s*[^\w\s]*\s*\]'
#     ]
#     for pattern in patterns:
#         text = re.sub(pattern, '', text, re.IGNORECASE)
#     return text

def handle_special_characters(text):
    """Handle special characters and Unicode normalization."""
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    char_map = {
        '´': "'", '‘': "'", '’': "'", '“': '"', '”': '"', '–': '-', '—': '-', '…': '...'
    }
    for char, replacement in char_map.items():
        text = text.replace(char, replacement)
    return text


def remove_references(text):
    # Define common markers for the start of a references section
    markers = [
        r'\bReferences\b',
        r'\bBibliography\b',
        r'\bWorks Cited\b',
        r'\bLiterature Cited\b'
    ]
    
    # Combine markers into a single pattern
    combined_pattern = re.compile(r'(' + '|'.join(markers) + r')\s*$', re.IGNORECASE)
    
    # Find all occurrences of the references markers
    matches = list(combined_pattern.finditer(text))
    
    if matches:
        # If matches are found, take the last one and remove everything after it
        text = text[:matches[-1].start()].strip()
    else:
        # If no specific markers, look for typical reference formatting (e.g., numbered references)
        reference_format_pattern = re.compile(r'\d+\.\s+[A-Za-z]+[\s\w,]*\.\s+\d{4};\d+\([\d\-]+\):\d+\-\d+', re.MULTILINE)
        matches = list(reference_format_pattern.finditer(text))
        
        if matches:
            # If multiple matches, assume the last one marks the start of the reference section
            text = text[:matches[-1].start()].strip()

    return text


def detect_sentence_boundaries(text):
    """
    Detect sentence boundaries using spaCy.
    """
    doc = nlp(text)
    spacy_sentences = [sent.text for sent in doc.sents]
    return spacy_sentences  # Return as a list of sentences

In [5]:
def count_words(text: str) -> int:
    return len(text.split())

In [6]:
def preprocess_text(text):
    try:
        # Input validation
        if not isinstance(text, (str, bytes)):
            raise TypeError('Expected a string or bytes-like object')

        print(f"Starting preprocessing... Initial word count: {count_words(text)}")

        # Step 1: Initial text cleaning
        cleaned_text = clean_pdf_text(text)
        print(f"Word count after clean_pdf_text: {count_words(cleaned_text)}")

        cleaned_text = remove_urls(cleaned_text)
        print(f"Word count after remove_urls: {count_words(cleaned_text)}")

        cleaned_text = remove_emails(cleaned_text)
        print(f"Word count after remove_emails: {count_words(cleaned_text)}")

        cleaned_text = remove_figures_tables(cleaned_text)
        print(f"Word count after remove_figures_tables: {count_words(cleaned_text)}")

        # cleaned_text = remove_numerical_references(cleaned_text)
        cleaned_text = remove_citations(cleaned_text)
        print(f"Word count after remove_citations: {count_words(cleaned_text)}")

        # cleaned_text = remove_headers(cleaned_text)
        # print(f"Word count after remove_headers: {count_words(cleaned_text)}")

        # cleaned_text = remove_metadata(cleaned_text)
        # print(f"Word count after remove_metadata: {count_words(cleaned_text)}")

        cleaned_text = handle_special_characters(cleaned_text)
        print(f"Word count after handle_special_characters: {count_words(cleaned_text)}")

        cleaned_text = remove_references(cleaned_text)
        print(f"Word count after remove_references: {count_words(cleaned_text)}")

        # Detect sentence boundaries and join sentences back into a single string
        cleaned_text = ' '.join(detect_sentence_boundaries(cleaned_text))
        print(f"Final word count after detect_sentence_boundaries: {count_words(cleaned_text)}")

        return cleaned_text

    except TypeError as te:
        logging.error(f"TypeError in preprocess_text: {te}")
        return None
    except Exception as e:
        logging.error(f"Error in preprocess_text: {e}")
        return None


In [7]:
def extract_abstract_and_main_text(cleaned_text):
    """Extracts the abstract from the text using refined regex patterns for length and structure."""
    # Patterns for potential abstract start
    abstract_start_patterns = [
        r'\b(Abstract|Synopsis|Summary|Overview)\b',  # Specific abstract headers
        r'^[A-Z][^.!?]*?\s+.*?[.!?]',  # Fallback for a capitalized sentence
    ]

    # Patterns for potential abstract end, prioritized
    abstract_end_patterns = [
        r'\b(Keywords|Key words)\b',  # Prioritize Keywords as the end of the abstract
        r'\bINTRODUCTION\b'           # Use Introduction if Keywords is not found
    ]

    abstract_start = None
    abstract_end = None

    # Find the start of the abstract
    for pattern in abstract_start_patterns:
        match = re.search(pattern, cleaned_text, re.IGNORECASE | re.MULTILINE)
        if match:
            abstract_start = match.start()
            break

    # If no explicit abstract start is found, fallback strategy
    if abstract_start is None:
        # Extract the first paragraph as a potential abstract
        paragraphs = re.split(r'\n{2,}', cleaned_text.strip())
        abstract = paragraphs[0].strip() + " (Assumed abstract)"
        return abstract, cleaned_text.strip()

    # Search for the end of the abstract, prioritizing "Keywords" over "Introduction"
    for pattern in abstract_end_patterns:
        match = re.search(pattern, cleaned_text[abstract_start:], re.IGNORECASE)
        if match:
            abstract_end = abstract_start + match.start()
            break

    # If no end is found, limit the abstract to 300 words
    if abstract_end is None:
        words = cleaned_text[abstract_start:].split()
        abstract_end = abstract_start + len(' '.join(words[:300]))

    # Extract the abstract
    abstract = cleaned_text[abstract_start:abstract_end].strip()

    # Extract the main text starting from the end of the abstract
    main_text = cleaned_text[abstract_end:].strip()

    return abstract, main_text

In [8]:
def extract_umls_entities(abstract):
    doc = nlp(abstract)
    entities = []
    for ent in doc.ents:
        if ent._.kb_ents:
            top_concept = ent._.kb_ents[0]
            entities.append({
                "text": ent.text,
                "label": ent.label_,
                "concept_id": top_concept[0],
                "score": top_concept[1],
                "start": ent.start_char,
                "end": ent.end_char
            })
    return entities

In [9]:
def process_files(input_folder_path: str, output_folder_path: str, entities_path: str, batch_size: int = 2) -> None:
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    if not os.path.exists(entities_path):
        os.makedirs(entities_path)

    files = [f for f in os.listdir(input_folder_path) if f.endswith('.txt')]
    total_files = len(files)

    all_entities = []
    word_counts = {}

    for i in range(0, total_files, batch_size):
        batch = files[i:i+batch_size]
        print(f"\nProcessing batch {i//batch_size + 1} of {(total_files-1)//batch_size + 1}")

        for filename in tqdm(batch, desc=f"Batch {i//batch_size + 1}"):
            input_file_path = os.path.join(input_folder_path, filename)
            output_file_path = os.path.join(output_folder_path, f"processed_{filename}")

            print(f"\nProcessing file: {filename}")

            try:
                with open(input_file_path, 'rb') as file:
                    raw_data = file.read()
                    result = chardet.detect(raw_data)
                    detected_encoding = result['encoding']
                    confidence = result['confidence']

                print(f"Detected encoding: {detected_encoding} with confidence: {confidence}")

                encoding_to_use = detected_encoding if detected_encoding else 'utf-8'

                with open(input_file_path, 'r', encoding=encoding_to_use) as file:
                    original_text = file.read()

                if not isinstance(original_text, str):
                    raise ValueError("File content is not a valid string.")

  

                # Count words in the cleaned text
                original_word_count = count_words(original_text)

                # Preprocess the original text
                cleaned_text = preprocess_text(original_text)

                # Extract abstract and main text using cleaned text
                abstract, main_text = extract_abstract_and_main_text(cleaned_text)

                abstract_entities = extract_umls_entities(abstract)
                all_entities.extend([{**entity, "filename": filename} for entity in abstract_entities])


                # Count words in the preprocessed text
                preprocessed_word_count = count_words(cleaned_text)



                # Prepare output text
                output_text = f"Abstract:\n{abstract}\n\nMain Text:\n{main_text}"


                print(f"Original Word Count: {original_word_count}")
                print(f"Preprocessed Word Count: {preprocessed_word_count}")
                print(f"Output text for {filename}: {output_text[:1000]}")

                # Write the cleaned text and abstract to the output file
                with open(output_file_path, 'w', encoding='utf-8') as file:
                    file.write(output_text)

                # Store word counts
                word_counts[filename] = {
                    "original": original_word_count,
                    "preprocessed": preprocessed_word_count
                }

            except Exception as e:
                logging.error(f"Error processing file {filename}: {str(e)}")

    # Save the extracted entities to a CSV file
    entities_df = pd.DataFrame(all_entities)
    entities_df.to_csv(os.path.join(entities_path, 'abstract_umls_entities.csv'), index=False)

    # Save word counts to a CSV file
    word_counts_df = pd.DataFrame.from_dict(word_counts, orient='index')
    word_counts_df.to_csv(os.path.join(output_folder_path, 'word_counts.csv'))

    print("\nProcessing completed.")

In [10]:
if __name__ == "__main__":
    # input_folder = "/mnt/data/skanda/MSc_IRD_LLM/data/txt_data"
    input_folder = "/mnt/scratch/skanda/MSc_IRD_LLM/data/txt_data"
    output_folder = "/mnt/scratch/skanda/MSc_IRD_LLM/data/data_preprocessed"
    entities_path = "/mnt/scratch/skanda/MSc_IRD_LLM/data/Entities/"
    process_files(input_folder, output_folder, entities_path, batch_size=2)
    print("Preprocessing and entity extraction complete.")


Processing batch 1 of 26


Batch 1:   0%|          | 0/2 [00:00<?, ?it/s]


Processing file: 1-s2.0-S1350946221000367-main.txt
Detected encoding: utf-8 with confidence: 0.99
Starting preprocessing... Initial word count: 21288
Word count after clean_pdf_text: 21001
Word count after remove_urls: 21000
Word count after remove_emails: 20999
Word count after remove_figures_tables: 20999
Word count after remove_citations: 20383
Word count after handle_special_characters: 20385
Word count after remove_references: 20385


Batch 1:  50%|█████     | 1/2 [00:04<00:04,  4.07s/it]

Final word count after detect_sentence_boundaries: 20385
Original Word Count: 21288
Preprocessed Word Count: 20385
Output text for 1-s2.0-S1350946221000367-main.txt: Abstract:
overview of existing modalities of retinal gene therapy and their relevance based on the needs of patient populations suffering from inherited retinal degenerations. 1.

Main Text:
Introduction 1.1. Why gene therapy? Why now? Gene therapy that is broadly defined as the use of exogenous DNA to treat human disease has been proposed as a potential medical intervention over five decades ago (Aposhian,  ). Gene therapy offers a theoretical advantage over small molecules classically used as medi - cines: it is a one-shot treatment that can provide a durable and poten - tially curative clinical benefit. The journey from concept to clinical application has been long, including the development of an experi - mental approach around the identified medical needs. The first appli - cations were evidently in vitro where cells 

...

Original Word Count: 6794
Preprocessed Word Count: 6390
Output text for fphar-12-654445.txt: Abstract:
summary, elevation of adenosine signaling represents a positive response of RPE cells to AMD. Muller cells intimately contact neurons and non-neural structures providing for uptake of neurotransmitters fromextracellular space, as well as controlling retinal potassium andwater homeostasis to maintain the extracellular space volume(,  ). Activation of P2X7 depolarizes Muller cells, thus decreasing the rate of the glial glutamate uptake resulting in elevated extracellular glutamate levels. Glutamatergic neurotransmission evokes ion currents whichlead to osmotic imbalances and a swelling of neuronal cellsbodies and synapses ( ,  ). Intense neuronal activity also decreases extracellular osmolarity due to theactivity-dependent decrease in extracellular sodium chloridewhich is approximately twice as large as the increase inextracellular potassium ( ,  ). Then, decreased extracellular osmolar

Batch 26:   0%|          | 0/1 [00:00<?, ?it/s]


Processing file: cells-12-02579-compressed.txt
Detected encoding: utf-8 with confidence: 0.99
Starting preprocessing... Initial word count: 15466
Word count after clean_pdf_text: 15294
Word count after remove_urls: 15293
Word count after remove_emails: 15291
Word count after remove_figures_tables: 15291
Word count after remove_citations: 15161
Word count after handle_special_characters: 15145
Word count after remove_references: 15145


Batch 26: 100%|██████████| 1/1 [00:02<00:00,  2.84s/it]

Final word count after detect_sentence_boundaries: 15146
Original Word Count: 15466
Preprocessed Word Count: 15146
Output text for cells-12-02579-compressed.txt: Abstract:
Abstract: Familial Exudative Vitreoretinopathy (FEVR), Norrie disease, and persistent fetal vascular syndrome (PFVS) are extremely rare retinopathies that are clinically distinct but are unified by abnormal retinal endothelial cell function, and subsequent irregular retinal vascular development and/or aberrant inner blood-retinal-barrier (iBRB) function. The early angiogenesis of the retina and its iBRB is a delicate process that is mediated by the canonical Norrin Wnt-signaling pathway in retinal endothelial cells. Pathogenic variants in genes that play key roles within this pathway, such as NDP, FZD4, TSPAN12, andLRP5, have been associated with the incidence of these retinal diseases. Recent efforts to further elucidate the etiology of these conditions have not only highlighted their multigenic nature but have also

In [11]:
def separate_abstract_and_main_text(preprocessed_text: str) -> tuple:
    """
    Separates the abstract and main text from the given preprocessed text.
    
    :param preprocessed_text: The input text containing both abstract and main text.
    :return: A tuple containing the abstract and main text as separate strings.
    """
    abstract_marker = "Abstract:"
    main_text_marker = "Main Text:"
    
    # Find the start of the abstract
    abstract_start = preprocessed_text.find(abstract_marker)
    # Find the start of the main text
    main_text_start = preprocessed_text.find(main_text_marker)
    
    # Extract abstract and main text
    if abstract_start != -1 and main_text_start != -1:
        abstract = preprocessed_text[abstract_start + len(abstract_marker):main_text_start].strip()
        main_text = preprocessed_text[main_text_start + len(main_text_marker):].strip()
    else:
        raise ValueError("The text does not contain the required markers for Abstract and Main Text.")
    
    return abstract, main_text

def process_preprocessed_files(input_folder_path: str, output_folder_path: str) -> None:
    """
    Processes all .txt files in the input folder, separates abstracts and main texts,
    and saves them into the output folder.
    
    :param input_folder_path: The path to the input folder containing .txt files.
    :param output_folder_path: The path to the output folder where separated files will be saved.
    """
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    # Get a list of all .txt files in the input directory
    files = [f for f in os.listdir(input_folder_path) if f.endswith('.txt')]
    
    for filename in files:
        input_file_path = os.path.join(input_folder_path, filename)
        print(f"Processing file: {filename}")
        
        try:
            # Read the content of the file
            with open(input_file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()
            
            # Separate abstract and main text
            abstract, main_text = separate_abstract_and_main_text(file_content)
            
            # Define paths for the abstract and main text files
            abstract_output_path = os.path.join(output_folder_path, f"{filename[:-4]}_abstract.txt")
            main_text_output_path = os.path.join(output_folder_path, f"{filename[:-4]}_main_text.txt")
            
            # Save the abstract
            with open(abstract_output_path, 'w', encoding='utf-8') as abstract_file:
                abstract_file.write(abstract)
            
            # Save the main text
            with open(main_text_output_path, 'w', encoding='utf-8') as main_text_file:
                main_text_file.write(main_text)
            
            print(f"Successfully processed and saved: {filename}")
        
        except Exception as e:
            print(f"Error processing file {filename}: {str(e)}")

input_folder = '/mnt/scratch/skanda/MSc_IRD_LLM/data/data_preprocessed'
output_folder = '/mnt/scratch/skanda/MSc_IRD_LLM/data/data_separated'

process_preprocessed_files(input_folder, output_folder)

Processing file: processed_41598_2021_Article_81093.txt
Successfully processed and saved: processed_41598_2021_Article_81093.txt
Processing file: processed_IJO-70-2316.txt
Successfully processed and saved: processed_IJO-70-2316.txt
Processing file: processed_main.txt
Successfully processed and saved: processed_main.txt
Processing file: processed_MGG3-9-e1663.txt
Successfully processed and saved: processed_MGG3-9-e1663.txt
Processing file: processed_13287_2023_Article_3526.txt
Successfully processed and saved: processed_13287_2023_Article_3526.txt
Processing file: processed_main copy.txt
Successfully processed and saved: processed_main copy.txt
Processing file: processed_nihms-1927912.txt
Successfully processed and saved: processed_nihms-1927912.txt
Processing file: processed_10.1177_2515841420954592.txt
Successfully processed and saved: processed_10.1177_2515841420954592.txt
Processing file: processed_main (1).txt
Successfully processed and saved: processed_main (1).txt
Processing file